In [ ]:
import requests

# Définir l'URL du endpoint et les headers
url = "https://aristote-dispatcher.mydocker-run-vd.centralesupelec.fr/v1/chat/completions"
headers = {
    "Authorization": "Bearer nothing",  # Remplacez "nothing" par votre clé API si nécessaire
    "Content-Type": "application/json"
}

# Préparer la requête JSON
payload = {
    "model": "casperhansen/llama-3-70b-instruct-awq",  # Nom du modèle
    "messages": [
        {"role": "system", "content": "You are a helpful assistant that translates English to French."},
        {"role": "user", "content": "I love programming."}
    ]
}

# Envoyer la requête
response = requests.post(url, headers=headers, json=payload)

# Vérifier la réponse
if response.status_code == 200:
    print("Réponse de l'API :")
    print(response.json()["choices"][0]["message"]["content"])
else:
    print("Erreur :", response.status_code, response.text)


In [4]:
# Configuration du prompt et appel à l'API Aristote via LangChain
system_prompt = """
You are an expert email classifier. Your task is to determine:
1. The main topic of the email (e.g., budget, compliance, campaign, etc.).
2. The department the email is related to (e.g., Finance, IT, Legal, Marketing, etc.).
Provide the results in this format:
Topic: <topic>
Department: <department>
"""


# Importation de la bibliothèque LangChain
from langchain_openai import ChatOpenAI

# Définir l'URL de l'API et le modèle
url = "https://aristote-dispatcher.mydocker-run-vd.centralesupelec.fr/v1"
llm = ChatOpenAI(
    base_url=url, 
    model="casperhansen/llama-3-70b-instruct-awq", 
    api_key="nothing"  # À remplacer si une clé API est requise
)

# Définir les messages pour la conversation
messages = [
    ("system", system_prompt),
    ("human", "Hello my name is Ilyes"),  # Message d'entrée de l'utilisateur
]

# Faire appel au LLM
response = llm.invoke(messages)

# Afficher la réponse
print("Réponse du modèle :")
print(response)


Réponse du modèle :
content='The username extracted from the message is: Ilyes' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 36, 'total_tokens': 48, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'casperhansen/llama-3-70b-instruct-awq', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-5eca956f-a77b-4607-b5df-027496186490-0' usage_metadata={'input_tokens': 36, 'output_tokens': 12, 'total_tokens': 48}


In [1]:
import pandas as pd 

df = pd.read_csv("top_nearest_clusters_maj.csv")
df.head()

,id,text,message
0,<32010299.1075861478364.JavaMail.evans@thyme>,lines emptive ty debt cash shares s dit rating...,Message-ID: <32010299.1075861478364.JavaMail.e...
1,<8776821.1075861476606.JavaMail.evans@thyme>,value bid companies americas broader inquiry c...,Message-ID: <8776821.1075861476606.JavaMail.ev...
2,<16798617.1075861476773.JavaMail.evans@thyme>,billion billion s stock plunge heavy blow mutu...,Message-ID: <16798617.1075861476773.JavaMail.e...
3,<4926311.1075861475898.JavaMail.evans@thyme>,formal inquiry party transactions s stock take...,Message-ID: <4926311.1075861475898.JavaMail.ev...
4,<6483993.1075861480647.JavaMail.evans@thyme>,tries investors busy signal edition ways portf...,Message-ID: <6483993.1075861480647.JavaMail.ev...


In [3]:
df_cleaned = df[df['text'].str.len() <= 300]
df_cleaned.count()

id         353
text       353
message    353
dtype: int64

In [3]:
df_10 = df.head(10)

In [4]:
df_cleaned_10 = df_cleaned.head(10)

In [6]:
import json
from langchain_openai import ChatOpenAI
from tqdm import tqdm

tqdm.pandas(desc="Analyse des e-mails")

# Configuration de l'API
url = "https://aristote-dispatcher.mydocker-run-vd.centralesupelec.fr/v1"
llm = ChatOpenAI(
    base_url=url,
    model="casperhansen/llama-3-70b-instruct-awq",
    api_key="nothing"  # Remplacez "nothing" si une clé API est requise
)

# Liste des e-mails
emails  = df_cleaned_10["message"].tolist()

# Prompt système
system_prompt = """
You are an expert email classifier. Your task is to determine:
1. The main topic of the email (e.g., decision-making, technical, business,legal clause etc.).
2. The department the email is related to (e.g., Finance, IT, Legal, Marketing, etc.).
Provide the results in this format:
{
    "topic": "<topic>",
    "department": "<department>"
}
"""

# Liste pour stocker les résultats
results = []

# Analyse des e-mails
for email in emails:
    messages = [
        ("system", system_prompt),
        ("human", email),
    ]
    response = llm.invoke(messages)
    
    # Vérifier et extraire le contenu textuel de la réponse
    if hasattr(response, "content"):
        response_content = response.content  # Accès au texte brut si nécessaire
    else:
        response_content = str(response)  # Convertir en chaîne si non structuré
    
    # Convertir la réponse en dict (si le modèle retourne un JSON-like string)
    try:
        classification = json.loads(response_content)
    except json.JSONDecodeError:
        print(f"Erreur de décodage JSON pour l'email : {email}")
        classification = {"topic": "N/A", "department": "N/A"}
    
    # Ajouter le texte de l'email dans le résultat
    classification["texte"] = email
    
    # Ajouter le résultat au tableau
    results.append(classification)

# Sauvegarder les résultats dans un fichier JSON
output_file = "email_classification_results.json"
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

print(f"Les résultats ont été sauvegardés dans le fichier : {output_file}")


Les résultats ont été sauvegardés dans le fichier : email_classification_results.json


In [7]:
import json
from langchain_openai import ChatOpenAI
from tqdm import tqdm

tqdm.pandas(desc="Analyse des e-mails")



def classify_email(email):

    # Configuration de l'API
    url = "https://aristote-dispatcher.mydocker-run-vd.centralesupelec.fr/v1"
    llm = ChatOpenAI(
        base_url=url,
        model="casperhansen/llama-3-70b-instruct-awq",
        api_key="nothing"  # Remplacez "nothing" si une clé API est requise
    )


    system_prompt = """
You are an expert email classifier. Your task is to determine:
1. The main topic of the email (e.g., decision-making, technical, business,legal clause etc.).
2. The department the email is related to (e.g., Finance, IT, Legal, Marketing, etc.).
Provide the results in this format:
{
    "topic": "<topic>",
    "department": "<department>"
}
"""
    messages = [
        ("system", system_prompt),
        ("human", email),
    ]
    response = llm.invoke(messages)
    
    # Vérifier et extraire le contenu textuel de la réponse
    if hasattr(response, "content"):
        response_content = response.content
    else:
        response_content = str(response)
    
    # Convertir la réponse en dict
    try:
        classification = json.loads(response_content)
    except json.JSONDecodeError:
        print(f"Erreur de décodage JSON pour l'email : {email}")
        classification = {"topic": "N/A", "department": "N/A"}
    
    classification["texte"] = email
    return classification

# Appliquer la classification avec une barre de progression
results = df_10["message"].progress_apply(classify_email)

# Convertir les résultats en DataFrame et sauvegarder
results_df = pd.DataFrame(results.tolist())
results_df.to_json("email_classification_results.json", orient="records", indent=4)

print("Les résultats ont été sauvegardés dans 'email_classification_results.json'.")


Analyse des e-mails: 100%|██████████| 10/10 [02:02<00:00, 12.22s/it]

Les résultats ont été sauvegardés dans 'email_classification_results.json'.


### Description de cluster basée sur les 10 emails centroides du clusters

In [6]:
df = pd.read_csv("top_nearest_clusters_maj_v6 (1).csv")
df.head()

,id,text,hdbscan_cluster,message
0,<32010299.1075861478364.JavaMail.evans@thyme>,lines emptive ty debt cash shares s dit rating...,1016,Message-ID: <32010299.1075861478364.JavaMail.e...
1,<8776821.1075861476606.JavaMail.evans@thyme>,value bid companies americas broader inquiry c...,1016,Message-ID: <8776821.1075861476606.JavaMail.ev...
2,<16798617.1075861476773.JavaMail.evans@thyme>,billion billion s stock plunge heavy blow mutu...,1016,Message-ID: <16798617.1075861476773.JavaMail.e...
3,<4926311.1075861475898.JavaMail.evans@thyme>,formal inquiry party transactions s stock take...,1016,Message-ID: <4926311.1075861475898.JavaMail.ev...
4,<6483993.1075861480647.JavaMail.evans@thyme>,tries investors busy signal edition ways portf...,1016,Message-ID: <6483993.1075861480647.JavaMail.ev...


In [9]:
import pandas as pd

# Grouper par la colonne 'hdbscan_cluster' et agréger
clustered_emails = df.groupby("hdbscan_cluster").agg({
    'id': list,       # Liste des IDs d'e-mails dans chaque cluster
    'message': list   # Liste des messages dans chaque cluster
}).reset_index()

# Afficher les 5 premières lignes
print(clustered_emails.head())

clustered_emails_100 = clustered_emails.head(10)


   hdbscan_cluster                                                 id  \
0                0  [<29188638.1075861489493.JavaMail.evans@thyme>...   
1                1  [<25543073.1075851585597.JavaMail.evans@thyme>...   
2             1000  [<32663381.1075851603967.JavaMail.evans@thyme>...   
3             1002  [<7345700.1075843350550.JavaMail.evans@thyme>,...   
4             1003  [<15412309.1075851600786.JavaMail.evans@thyme>...   

                                             message  
0  [Message-ID: <29188638.1075861489493.JavaMail....  
1  [Message-ID: <25543073.1075851585597.JavaMail....  
2  [Message-ID: <32663381.1075851603967.JavaMail....  
3  [Message-ID: <7345700.1075843350550.JavaMail.e...  
4  [Message-ID: <15412309.1075851600786.JavaMail....  


In [ ]:
import pandas as pd
import json
from langchain_openai import ChatOpenAI
from tqdm import tqdm

tqdm.pandas(desc="Analyse des clusters")


# Configuration de l'API
url = "https://aristote-dispatcher.mydocker-run-vd.centralesupelec.fr/v1"
llm = ChatOpenAI(
    base_url=url,
    model="casperhansen/llama-3-70b-instruct-awq",
    api_key="nothing"  # Remplacez "nothing" si une clé API est requise
)

# Prompt système
system_prompt = """
You are an expert in text summarization and analysis. Your task is to analyze the provided emails and:
1. Identify the main topic or common themes of the cluster (e.g., business strategy, technical issues, legal disputes, marketing campaigns, etc.).
2. Provide a concise description of the cluster in 2-3 sentences summarizing the purpose or key focus of the emails.
The input emails are related and belong to the same group. Focus on their commonalities.
Return the results in this format:
{
    "description": "<description>",
    "common_topic": "<common_topic>"
}
"""

# Liste pour stocker les résultats
cluster_results = []

# Analyse des clusters
for row in tqdm(clustered_emails.iterrows(), desc="Processing clusters"):
    # Récupérer les e-mails et leurs IDs pour le cluster courant
    email_texts = row["message"]
    email_ids = row["email_id"]
    
    # Préparer les e-mails pour le prompt
    batch_text = "\n\n".join([f"Email {i+1}: {email}" for i, email in enumerate(email_texts, start=1)])
    batch_prompt = f"The following emails belong to cluster {cluster_id}:\n\n{batch_text}"
    
    # Construire les messages pour le LLM
    messages = [
        ("system", system_prompt),
        ("human", batch_prompt),
    ]
    
    # Appeler le LLM
    response = llm.invoke(messages)
    
    # Vérifier et extraire le contenu textuel de la réponse
    if hasattr(response, "content"):
        response_content = response.content  # Accès au texte brut si nécessaire
    else:
        response_content = str(response)  # Convertir en chaîne si non structuré
    
    # Convertir la réponse en dict (si le modèle retourne un JSON-like string)
    try:
        cluster_summary = json.loads(response_content)
    except json.JSONDecodeError:
        print(f"Erreur de décodage JSON pour le cluster {cluster_id}")
        cluster_summary = {"description": "N/A", "common_topic": "N/A"}
    
    # Ajouter le numéro de cluster et la liste des IDs des e-mails
    cluster_summary["cluster_id"] = cluster_id
    cluster_summary["email_ids"] = email_ids
    
    # Ajouter le résultat au tableau
    cluster_results.append(cluster_summary)

# Sauvegarder les résultats dans un fichier JSON
output_file = "cluster_descriptions.json"
with open(output_file, "w") as f:
    json.dump(cluster_results, f, indent=4)

print(f"Les descriptions des clusters ont été sauvegardées dans le fichier : {output_file}")


In [12]:
import pandas as pd
import json
from langchain_openai import ChatOpenAI
from tqdm import tqdm

# Configuration de l'API
url = "https://aristote-dispatcher.mydocker-run-vd.centralesupelec.fr/v1"
llm = ChatOpenAI(
    base_url=url,
    model="casperhansen/llama-3-70b-instruct-awq",
    api_key="nothing"  # Remplacez "nothing" si une clé API est requise
)

# Prompt système
system_prompt = """
You are an expert in text summarization and analysis. Your task is to analyze the provided emails and:
1. Identify the main topic or common themes of the cluster (e.g., business strategy, technical issues, legal disputes, marketing campaigns, etc.).
2. Provide a concise description of the cluster in 2-3 sentences summarizing the purpose or key focus of the emails.
The input emails are related and belong to the same group. Focus on their commonalities.
Return the results in this format:
{
    "description": "<description>",
    "common_topic": "<common_topic>"
}
"""

# Liste pour stocker les résultats
cluster_results = []

# Analyse des clusters
for _, row in tqdm(clustered_emails_100.iterrows(), total=clustered_emails_100.shape[0], desc="Processing clusters"):
    cluster_id = row["hdbscan_cluster"]
    email_texts = row["message"]
    email_ids = row["id"]
    
    # Préparer les e-mails pour le prompt
    batch_text = "\n\n".join([f"Email {i+1}: {email}" for i, email in enumerate(email_texts, start=1)])
    batch_prompt = f"The following emails belong to cluster {cluster_id}:\n\n{batch_text}"
    #print(batch_prompt)
    # Construire les messages pour le LLM
    messages = [
        ("system", system_prompt),
        ("human", batch_prompt),
    ]
    
    try:
        # Appeler le LLM
        response = llm.invoke(messages)

        # Vérifier la réponse
        if response is None or not hasattr(response, "content"):
            print(f"Empty response for cluster {cluster_id}. Skipping...")
            continue

        response_content = response.content

        # Convertir la réponse en dict (si le modèle retourne un JSON-like string)
        try:
            cluster_summary = json.loads(response_content)
        except json.JSONDecodeError:
            print(f"Erreur de décodage JSON pour le cluster {cluster_id}")
            cluster_summary = {"description": "N/A", "common_topic": "N/A"}

    except Exception as e:
        print(f"Erreur lors de la requête pour le cluster {cluster_id}: {e}")
        cluster_summary = {"description": "N/A", "common_topic": "N/A"}
    
    # Ajouter le numéro de cluster et la liste des IDs des e-mails
    cluster_summary["cluster_id"] = cluster_id
    cluster_summary["email_ids"] = email_ids
    
    # Ajouter le résultat au tableau
    cluster_results.append(cluster_summary)

# Sauvegarder les résultats dans un fichier JSON
output_file = "cluster_descriptions.json"
with open(output_file, "w") as f:
    json.dump(cluster_results, f, indent=4)

print(f"Les descriptions des clusters ont été sauvegardées dans le fichier : {output_file}")


Processing clusters:  10%|█         | 1/10 [00:00<00:02,  3.32it/s]

Erreur lors de la requête pour le cluster 0: 'NoneType' object is not iterable


Processing clusters:  20%|██        | 2/10 [00:00<00:02,  3.60it/s]

Erreur lors de la requête pour le cluster 1: 'NoneType' object is not iterable


Processing clusters: 100%|██████████| 10/10 [02:04<00:00, 12.42s/it]

Les descriptions des clusters ont été sauvegardées dans le fichier : cluster_descriptions.json


In [10]:
# Analyse des clusters
from tqdm import tqdm
for _, row in tqdm(clustered_emails_100.iterrows(), total=clustered_emails_100.shape[0], desc="Processing clusters"):
    cluster_id = row["hdbscan_cluster"]
    email_texts = row["message"]
    email_ids = row["id"]
    
    # Préparer les e-mails pour le prompt
    batch_text = "\n\n".join([f"Email {i+1}: {email}" for i, email in enumerate(email_texts, start=1)])
    batch_prompt = f"The following emails belong to cluster {cluster_id}:\n\n{batch_text}"
    print(batch_prompt)

Processing clusters: 100%|██████████| 10/10 [00:00<00:00, 2496.46it/s]

The following emails belong to cluster 0:

Email 2: Message-ID: <29188638.1075861489493.JavaMail.evans@thyme>
Date: Tue, 27 Nov 2001 04:55:01 -0800 (PST)
From: alert@venturewire.com
To: jdasovic@enron.com
Subject: VentureWire Alert, Tuesday, November 27, 2001
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: VentureWire Alert <alert@venturewire.com>
X-To: jdasovic@ENRON.COM
X-cc: 
X-bcc: 
X-Folder: \JDASOVIC (Non-Privileged)\Dasovich, Jeff\Deleted Items
X-Origin: Dasovich-J
X-FileName: JDASOVIC (Non-Privileged).pst

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<html>
<head>
<style type="text/css">
A:link {text-decoration:none; color: GRAY;}
A:visited {text-decoration:none; color: dimgray;}
A:hover {text-decoration:none; color: tomato;}
A.search {font-size:8pt;}
</style>
<title>VentureWire Alert</title>
<meta HTTP-EQUIV="Content-Type" CONTENT="text/html;charset=iso-8859-1">
<!--
If you are reading this, your e-mail
cli